### NOTE: you need to set your SOLANARPC_HTTP_URI env var

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
from external_klend_utils import *
import time
import os
from pprint import pprint

# Fetch Current Loan Data from On-Chain

In [4]:
async def get_all_market_data(market):
    start_time = time.time()
    
    uri = os.getenv("SOLANARPC_HTTP_URI")

    # make kamino_lend_program
    kamino_lend_program = get_kamino_lend_program(uri)

    reserves_configs = await get_reserves_configs(kamino_lend_program, market)
    elevation_group_dict = await get_elevation_group_dict(market)
    loan_metrics_df = await get_loans_metrics(reserves_configs, elevation_group_dict, lending_market=market)

    time_taken = time.time() - start_time
    logging.info(f"time_taken = {time_taken:.2f}s")
    
    print(f"Number of reserves = {len(reserves_configs)}")
    print(f"Number of elevation groups = {len(elevation_group_dict)}")
    print(f"Number of loans = {loan_metrics_df.shape[0]:,}")
    return reserves_configs, elevation_group_dict,loan_metrics_df


    
uri = os.getenv("SOLANARPC_HTTP_URI")
# make kamino_lend_program
kamino_lend_program = get_kamino_lend_program(uri)


In [5]:
obligations = await get_all_obligations(
    kamino_lend_program,
    LENDING_MARKETS["ethena_market"]
)

inside


In [15]:
def object_to_dict(obj):
    """
    Exploding objects
    """
    print("inside")
    if isinstance(obj, list):
        print("here")
        return [object_to_dict(item) for item in obj]
    elif hasattr(obj, "__dict__"):
        result = {}
        for key, val in obj.__dict__.items():
            # Fixing this to not contain all of the padded values
            # some padding starts with padding others are config_padding etc
            print(f"key: {key}")
            if "padding" in str(key):
                print("skipping")
                continue
            element = []
            if isinstance(val, list):
                print("not skipping")
                for item in val:
                    if hasattr(item, "__dict__"):
                        element.append(object_to_dict(item))
                    else:
                        element.append(item)
                result[key] = element
            else:
                if hasattr(val, "__dict__"):
                    result[key] = object_to_dict(val)
                else:
                    result[key] = val
        return result
    else:
        return obj

In [16]:
dict = object_to_dict(obligations)

inside


In [ ]:
dic[Pubkey.from_string("Hgd2j52r71SFDyHZB3DjMPccb31yc3wes7JbUaGpQxBi")]

In [ ]:
kamino_client = KaminoClient(os.getenv("SOLANARPC_HTTP_URI"))
lending_market_object = await kamino_client.fetch_with_retries(
    LendingMarket.fetch, Pubkey.from_string("BJnbcRHqvppTyGesLzWASGKnmnF1wq9jZu6ExrjT7wvF"), extra=None
)


In [ ]:
lending_market_object.elevation_groups

In [ ]:
uri = os.getenv("SOLANARPC_HTTP_URI")
# make kamino_lend_program
kamino_lend_program = get_kamino_lend_program(uri)

elevation = await get_elevation_group_dict( "7u3HeHxYDLhnCoErrtycNokbQYbWGzLs6JSDqGAv5PfF")

In [ ]:
elevation

In [ ]:
elevation

In [ ]:
reserves_configs = await get_reserves_configs(kamino_lend_program, "7u3HeHxYDLhnCoErrtycNokbQYbWGzLs6JSDqGAv5PfF")

In [ ]:
reserves_configs

In [ ]:
reserves_configs['EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v']['account']['config']

In [ ]:
elevation

In [ ]:
lending_market = LENDING_MARKETS['ethena_market']
reserves_configs, elevation_group_dict,loan_metrics_df = await get_all_market_data(lending_market)

In [ ]:
for x in loan_metrics_df.keys():
    print(x)

In [ ]:
main_market_addr = "7u3HeHxYDLhnCoErrtycNokbQYbWGzLs6JSDqGAv5PfF"

In [ ]:
elevation_groups = await get_elevation_group_dict(LENDING_MARKETS['main_market'])

In [ ]:
LENDING_MARKETS['main_market']

In [ ]:
elevation_groups

In [ ]:
reserves_configs['J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn']['account']['config']


In [ ]:
kamino_client = KaminoClient(os.getenv("SOLANARPC_HTTP_URI"))
lending_market_object = await kamino_client.fetch_with_retries(
    LendingMarket.fetch, Pubkey.from_string(LENDING_MARKETS['main_market']), extra=None
)

In [ ]:
Pubkey

In [ ]:
await LendingMarket.fetch(client, Pubkey.from_string(main_market_addr))

In [ ]:
    
uri = os.getenv("SOLANARPC_HTTP_URI")
# make kamino_lend_program
kamino_lend_program = get_kamino_lend_program(uri)

reserves = await get_reserves_configs(kamino_lend_program, main_market_addr)

In [ ]:

uri = os.getenv("SOLANARPC_HTTP_URI")

    # make kamino_lend_program
kamino_lend_program = get_kamino_lend_program(uri)
reserves_configs = await get_reserves_configs(kamino_lend_program, "7u3HeHxYDLhnCoErrtycNokbQYbWGzLs6JSDqGAv5PfF")

In [ ]:
print(reserves_configs)

In [ ]:
await kamino_lend_program.account["Reserve"].all()

In [ ]:
obligations = await kamino_lend_program.account["Obligation"].all()

In [ ]:
print(obligations[0])

In [ ]:
lending_markets = await kamino_lend_program.account["LendingMarket"].all(
    filters = []
)

In [ ]:
client = AsyncClient(os.getenv("SOLANARPC_HTTP_URI"))

In [ ]:
data = await client.get_account_info_json_parsed(
    Pubkey.from_string("Hy2S5arXGFgsvze47PCgSjF92ZQCiAfUJnFkqZQMXu4T")
)
dec = Reserve.layout.parse(data.value.data[ACCOUNT_DISCRIMINATOR_SIZE:])

In [ ]:
dec.config

In [ ]:
data = json.loads(data.to_json())

In [ ]:
dec = LendingMarket.layout.parse(data.value.data[ACCOUNT_DISCRIMINATOR_SIZE:])

In [ ]:
dec

In [ ]:
decoded_bytes = base64.b64decode(data['result']['value']['data'][0])
fir_8 = decoded_bytes[:8]
rep = fir_8.hex()
print(rep)

In [ ]:
bytes.fromhex(rep)

In [ ]:
len(rep)

In [ ]:
bytes.fromhex("5ae6f30c297b461d")

In [ ]:
Reserve.discriminator.hex()

In [ ]:
data['result']['value']['data']

In [ ]:
reserves = await kamino_lend_program.account["Reserve"].all(
    filters = [RESERVE_SIZE, MemcmpOpts(0, Reserve.discriminator)]
)

In [ ]:
Reserve.discriminator.hex()


In [ ]:
Obligation.discriminator.hex()

In [ ]:
elevation_groups

In [ ]:
from pprint import pprint as print
print('')
#
print(reserves_configs['J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn']['account']['config'].keys())

In [ ]:
elevation_groups

In [ ]:
mint_to_str_map, str_to_mint_map = await get_scope_mints_to_str_map()

In [ ]:
# token reserve tickers in this market
[mint_to_str_map[k] for k in reserves_configs.keys()]

In [ ]:
# token reserve tickers in this market which are shown in UI (status == 0)
reserves_configs_keys_ui = {
    k: r
    for k, r in reserves_configs.items()
    if r["account"]["config"]["status"] == 0
}

[mint_to_str_map[k] for k in reserves_configs_keys_ui.keys()]

In [ ]:
# view key cols
loan_metrics_df_copy = loan_metrics_df.copy()
cols = [
    "owner",
    "elevation_group",
    "current_ltv",
    "max_ltv",
    "unhealthy_ltv",
    "dist_to_liq",
    "total_deposit_usd",
    "total_borrow_usd",
    "net_value",
]
loan_metrics_df_copy = loan_metrics_df_copy[cols]
loan_metrics_df_copy = loan_metrics_df_copy[loan_metrics_df_copy.total_borrow_usd > 0.0]
loan_metrics_df_copy.sort_values(by='dist_to_liq', ascending=True, inplace=True)
loan_metrics_df_copy.head(5)

In [ ]:
# get IR curves
interest_rates = {}
for token in reserves_configs.keys():
    (
        df_interpolated,
        borrow_curve_df,
        current_util,
        current_borrow_rate,
        current_borrow_rate_apy,
        current_supply_rate,
        current_supply_rate_apy,
    ) = get_ir_curve(reserves_configs, token)
    interest_rates[token] = {
        "df_interpolated": df_interpolated,
        "borrow_curve_df": borrow_curve_df,
        "current_util": current_util,
        "current_borrow_rate": current_borrow_rate,
        "current_borrow_rate_apy": current_borrow_rate_apy,
        "current_supply_rate": current_supply_rate,
        "current_supply_rate_apy": current_supply_rate_apy,
    }
interest_rates.keys()

In [ ]:
interest_rates['EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v']['borrow_curve_df']